In [ ]:
from groq import Groq
from duckduckgo_search import DDGS
from py_expression_eval import Parser
import re, time
from datetime import datetime

# Initialize Groq client
client = Groq()

# Initialize calculator
parser = Parser()

# Tool functions
def search(search_term):
    search_result = ""
    with DDGS() as ddgs:
        results = ddgs.text(search_term, max_results=5)
        for result in results:
            search_result += result['body'] + " "
    return search_result

def calculator(str):
    return parser.parse(str).evaluate({})

def get_current_datetime(format_string=None):
    current = datetime.now()
    if format_string:
        try:
            return current.strftime(format_string)
        except ValueError:
            return str(current)
    return str(current)

# Add a new tool for character counting
def char_counter(text, char):
    count = 0
    result = []
    for c in text:
        if c == char:
            count += 1
        result.append(f"char={c}, count={count}")
    return f"Step-by-step counting:\n" + "\n".join(result) + f"\n\nFinal count: {count}"

# Get current date and time for system prompt
CURRENT_DATETIME = get_current_datetime()

# Improved system prompt with observation format
SYSTEM_PROMPT = f"""
You are an AI assistant with access to current information. The current date and time is: {CURRENT_DATETIME}

You have access to the following tools:

1. Search: Useful for when you need to answer questions about current events. You should ask targeted questions.
2. Calculator: Useful for when you need to answer questions about math. Use python code, eg: 2 + 2
3. DateTime: Useful for when you need to know the current date or time in a specific format. You can provide a format string like "%Y-%m-%d" or "%H:%M:%S".
4. CharCounter: Counts occurrences of a character in text, showing step-by-step progress.
5. Response To Human: When you need to respond to the human you are talking to.

When processing a query, you should follow this specific format:

1. Thought: First, think about what you need to do to answer the question.
2. Action: Decide which tool to use - one of [Search, Calculator, DateTime].
3. Action Input: Provide the specific input for the chosen tool.
4. Observation: After using a tool, you'll receive an observation. Review this carefully.
5. Repeat steps 1-4 if needed, using the observation to inform your next action.
6. Once you have enough information, use:
   Action: Response To Human
   Action Input: "your final response, incorporating what you learned"

Example format:
Thought: I need to know the current temperature to compare it with yesterday's.
Action: Search
Action Input: "current temperature New York City"
Observation: [You will receive search results here]
Thought: Now I need yesterday's temperature to compare.
Action: Search
Action Input: "yesterday's temperature New York City"
Observation: [You will receive search results here]
Thought: I have all the information I need to respond.
Action: Response To Human
Action Input: "The current temperature in New York City is X, which is Y degrees different from yesterday's temperature of Z."

Remember:
- Always think before acting
- Use observations to guide your next steps
- You may need multiple tool uses before giving a final response
- Be precise in your action inputs
- When using DateTime, you can provide format strings for specific formats

Current date and time reference: {CURRENT_DATETIME}

Begin!
"""

def get_streaming_response(messages):
    completion = client.chat.completions.create(
        model="llama-3.2-11b-text-preview",
        messages=messages,
        temperature=0,
        max_tokens=1024,
        top_p=1,
        stream=True,
        stop=None,
    )
    
    full_response = ""
    for chunk in completion:
        content = chunk.choices[0].delta.content or ""
        full_response += content
        print(content, end="")
    print()  # New line after response
    return full_response

def stream_agent(prompt):
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": prompt},
    ]
    
    def extract_action_and_input(text):
        action_pattern = r"Action: (.+?)\n"
        input_pattern = r"Action Input: \"(.+?)\""
        action = re.findall(action_pattern, text)
        action_input = re.findall(input_pattern, text)
        return action, action_input
    
    while True:
        response_text = get_streaming_response(messages)
        
        action, action_input = extract_action_and_input(response_text)
        
        if not action or not action_input:
            break
            
        if action[-1] == "Search":
            tool = search
        elif action[-1] == "Calculator":
            tool = calculator
        elif action[-1] == "DateTime":
            tool = get_current_datetime
        elif action[-1] == "Response To Human":
            break
            
        observation = tool(action_input[-1])
        print("\nObservation: ", observation, "\n")
        
        messages.extend([
            {"role": "assistant", "content": response_text},
            {"role": "user", "content": f"Observation: {observation}"},
        ])

# Example usage
if __name__ == "__main__":
    test_queries = [
        "What's the temperature difference between Bengalurur and London right now?",
        "How many days until the next New Year?",
        "What was the top news story yesterday?"
    ]
    
    for query in test_queries:
        print(f"\n\nQuery: {query}")
        stream_agent(query)
        time.sleep(2)  # Pause between queries

In [ ]:
from groq import Groq
from duckduckgo_search import DDGS
from py_expression_eval import Parser
import re, time
from datetime import datetime

# Initialize Groq client
client = Groq()

# Initialize calculator
parser = Parser()

# DuckDuckGo search function
def search(search_term):
    search_result = ""
    with DDGS() as ddgs:
        results = ddgs.text(search_term, max_results=5)
        for result in results:
            search_result += result['body'] + " "
    return search_result

# Calculator function
def calculator(str):
    return parser.parse(str).evaluate({})

# DateTime function
def get_current_datetime(format_string=None):
    current = datetime.now()
    if format_string:
        try:
            return current.strftime(format_string)
        except ValueError:
            return str(current)
    return str(current)

# Get current date and time for system prompt
CURRENT_DATETIME = get_current_datetime()

# Improved system prompt
SYSTEM_PROMPT = f"""
You are an AI assistant with access to current information. The current date and time is: {CURRENT_DATETIME}

You have access to the following tools:

1. Search: Useful for when you need to answer questions about current events. You should ask targeted questions.
2. Calculator: Useful for math calculations. For counting characters, do it manually and show your work.
3. DateTime: Useful for when you need to know the current date or time in a specific format. You can provide a format string like "%Y-%m-%d" or "%H:%M:%S".
4. Response To Human: When you need to respond to the human you are talking to.

When processing a query, you should follow this specific format:

1. Thought: First, think about what you need to do. For counting or detailed analysis, plan to do it step by step.
2. Action: Decide which tool to use - one of [Search, Calculator, DateTime].
3. Action Input: Provide the specific input for the chosen tool.
4. Observation: After using a tool, you'll receive an observation. Review this carefully.
5. Thought: Analyze the observation. For counting tasks, always verify manually:
   - Go through each character one by one
   - Keep a running count
   - Show your work clearly
6. Repeat steps 1-5 if needed.
7. Final Response: Use "Response To Human" with verified information.

Example for careful counting:
Human: How many 'r's are in "strawberry"?
Thought: I need to count the occurrences of 'r' in "strawberry". I'll do this carefully, character by character.
s - No 'r', count remains 0
t - No 'r', count remains 0
r - Found first 'r', count is now 1
a - No 'r', count remains 1
w - No 'r', count remains 1
b - No 'r', count remains 1
e - No 'r', count remains 1
r - Found second 'r', count is now 2
r - Found third 'r', count is now 3
y - No 'r', count remains 3
Action: Response To Human
Action Input: "There are 3 'r's in the word 'strawberry'. I counted them one by one: the first 'r' comes after 'st', and then there are two more 'r's together near the end."

Example format:
Thought: I need to know the current temperature to compare it with yesterday's.
Action: Search
Action Input: "current temperature New York City"
Observation: [You will receive search results here]
Thought: Now I need yesterday's temperature to compare.
Action: Search
Action Input: "yesterday's temperature New York City"
Observation: [You will receive search results here]
Thought: I have all the information I need to respond.
Action: Response To Human
Action Input: "The current temperature in New York City is X, which is Y degrees different from yesterday's temperature of Z."

Remember:
- Always think before acting
- Use observations to guide your next steps
- You may need multiple tool uses before giving a final response
- Be precise in your action inputs
- When using DateTime, you can provide format strings for specific formats
- Always verify counts and calculations manually
- Show your step-by-step work for transparency
- Double-check before giving final answers
- For counting tasks, go through each item one by one

Begin!
"""

def get_streaming_response(messages):
    completion = client.chat.completions.create(
        model="llama-3.2-11b-text-preview",
        messages=messages,
        temperature=0,
        max_tokens=1024,
        top_p=1,
        stream=True,
        stop=None,
    )
    
    full_response = ""
    for chunk in completion:
        content = chunk.choices[0].delta.content or ""
        full_response += content
        print(content, end="")
    print()  # New line after response
    return full_response

def stream_agent(prompt):
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": prompt},
    ]
    
    def extract_action_and_input(text):
        action_pattern = r"Action: (.+?)\n"
        input_pattern = r"Action Input: \"(.+?)\""
        action = re.findall(action_pattern, text)
        action_input = re.findall(input_pattern, text)
        return action, action_input
    
    while True:
        response_text = get_streaming_response(messages)
        
        action, action_input = extract_action_and_input(response_text)
        
        if not action or not action_input:
            break
            
        if action[-1] == "Search":
            tool = search
        elif action[-1] == "Calculator":
            tool = calculator
        elif action[-1] == "DateTime":
            tool = get_current_datetime
        elif action[-1] == "Response To Human":
            break
            
        observation = tool(action_input[-1])
        print("\nObservation:", observation, "\n")
        
        messages.extend([
            {"role": "assistant", "content": response_text},
            {"role": "user", "content": f"Observation: {observation}"},
        ])

# Example usage
if __name__ == "__main__":
    test_queries = [
        # "How many 'r's are in the word 'strawberry'?",
        # "What's the current temperature in New York City?",
        # "Calculate 15% of 85",
        # "What day of the week will it be 3 days from now?"
    ]
    
    for query in test_queries:
        print(f"\n\nQuery: {query}")
        stream_agent(query)
        time.sleep(2)  # Pause between queries